In [1]:
import pandas as pd

In [ ]:
%pip install tweepy langdetect

In [3]:
import tweepy

client = tweepy.Client(bearer_token="AAAAAAAAAAAAAAAAAAAAAIHQcQEAAAAAltFqxFhiitQ6AT8R9LP8mX%2F9y8w%3DSTDzL94lipFbvKOqaLMNTFQKx2Tf0Y3fODsrwq0bjSkggdF5fY")

tweets = client.search_recent_tweets("(#bitcoin OR #Bitcoin OR #BTC OR #btc OR #cryptocurrencies) -is:retweet lang:en",max_results=100,).data
tweets_df = pd.DataFrame([tweet.data for tweet in tweets])
tweets_df.head()

,id,text
0,1565631262760501248,Deer coin fast in jumping the crypto \n#BNB #B...
1,1565631260655239171,"@B055Lady_Elle @POODLETOKEN First dominant, un..."
2,1565631259694743552,"@facugambande First dominant, unique and commu..."
3,1565631258188734464,@chaeiyo_0 @RichQuack The King Of Memes is her...
4,1565631257509249024,#btc @Acyn: 'McCarthy: The electric cord of li...


In [4]:
import re
def cleanTweets(txt):
    txt = str(txt)
    txt = re.sub(r'#','',txt)
    txt = re.sub(r'\n','',txt)
    txt = re.sub(r'https?:\/\/\S+','',txt)
    txt = re.sub(r'@','',txt)
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', txt).lower()

In [5]:
tweets_df.text = tweets_df.text.apply(cleanTweets)
tweets_df.head()

,id,text
0,1565631262760501248,deer coin fast in jumping the crypto bnb binan...
1,1565631260655239171,"b055lady_elle poodletoken first dominant, uniq..."
2,1565631259694743552,"facugambande first dominant, unique and commun..."
3,1565631258188734464,chaeiyo_0 richquack the king of memes is here ...
4,1565631257509249024,btc acyn: 'mccarthy: the electric cord of libe...


# FinBERT

In [8]:
%pip install -U torch transformers

     |█████▍                          | 129.4 MB 14.3 MB/s eta 0:00:46  |                                | 624 kB 4.1 MB/s eta 0:03:09     |▍                               | 9.5 MB 124 kB/s eta 1:42:31     |▌                               | 11.0 MB 134 kB/s eta 1:34:57     |▌                               | 11.4 MB 134 kB/s eta 1:34:55     |▊                               | 17.0 MB 134 kB/s eta 1:34:13     |█▍                              | 34.4 MB 125 kB/s eta 1:38:48     |█▊                              | 42.7 MB 122 kB/s eta 1:39:58     |██▎                             | 54.6 MB 5.1 MB/s eta 0:02:22     |██▍                             | 58.5 MB 5.1 MB/s eta 0:02:22     |███▏                            | 77.0 MB 124 kB/s eta 1:33:46     |███▏                            | 77.5 MB 124 kB/s eta 1:33:42     |███▌                            | 85.2 MB 121 kB/s eta 1:34:31     |███▉                            | 93.7 MB 6.9 MB/s eta 0:01:39     |████                            | 95.4 MB 24.

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import tqdm

# create a tokenizer object
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

# fetch the pretrained model 
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

def sentim_analyzer(df, tokenizer, model):
    ''' Given a df that contains a column 'headline' with article healine texts, it runs inference on the healine with the 'model' (FinBert) 
       and inserts output sentiment features into the dataframe in the respective columns (Positive_sentim, Negative_sentim, Neutral_sentim)
       
        Parameters :
          df : A dataframe that contains headlines in a column called 'headline' . 
          tokenizer(AutoTokenizer object) : A pre-processing tokenizer object from Hugging Face lib. 
          model (AutoModelForSequenceClassification object) : A hugging face transformer model.     
          
          returns df : The initial dataframe with the 3 sentiment features as columns for each headline'''
    
    for i in df.index :
        try:
            headline = df.loc[i, 'text']
        except:
            return print(' \'headline\' column might be missing from dataframe')
        # Pre-process input phrase
        input = tokenizer(headline, padding = True, truncation = True, return_tensors='pt')
        # Estimate output
        output = model(**input)
        # Pass model output logits through a softmax layer.
        predictions =  torch.nn.functional.softmax(output.logits, dim=-1)
        df.loc[i, 'Positive'] = predictions[0][0].tolist()
        df.loc[i, 'Negative'] = predictions[0][1].tolist()
        df.loc[i, 'Neutral']  = predictions[0][2].tolist()
    # rearrange column order
    try:
        df = df[['date', 'stock', 'Open', 'Close', 'Volume',  'headline', 'Positive', 'Negative', 'Neutral','Price_change']]
    except:
        pass
    return df

In [15]:
sentim_analyzer(tweets_df, tokenizer, model)

,id,text,Positive,Negative,Neutral
0,1565631262760501248,deer coin fast in jumping the crypto bnb binan...,0.212210,0.018872,0.768918
1,1565631260655239171,"b055lady_elle poodletoken first dominant, uniq...",0.324472,0.007769,0.667759
2,1565631259694743552,"facugambande first dominant, unique and commun...",0.395163,0.007597,0.597240
3,1565631258188734464,chaeiyo_0 richquack the king of memes is here ...,0.218423,0.010883,0.770694
4,1565631257509249024,btc acyn: 'mccarthy: the electric cord of libe...,0.123619,0.015842,0.860539
...,...,...,...,...,...
95,1565630999769550860,"shannonsheriff1 first dominant, unique and com...",0.354394,0.007740,0.637866
96,1565630998460518400,airdropstario great project to earn $kiss. tha...,0.378834,0.009263,0.611903
97,1565630998418804736,siksparjonaks1 solrarity_ y00tsnft obendersmar...,0.285853,0.010816,0.703331
98,1565630997982597120,martiniguyyt gm. make your smartest financial ...,0.046597,0.022341,0.931062


In [21]:
import numpy as np 
tweets_df['Sentiment_pos_neg']=tweets_df.apply(lambda x: 'Positive' if x['Positive']>x['Negative'] else 'Negative', axis=1)
tweets_df['Sentiment']=tweets_df.apply(lambda x: ['Positive','Negative','Neutral'][np.array([x['Positive'], x['Negative'], x['Neutral']]).argmax()], axis=1)

In [23]:
tweets_df.Sentiment_pos_neg.value_counts()

Positive    93
Negative     7
Name: Sentiment_pos_neg, dtype: int64

In [24]:
tweets_df.Sentiment.value_counts()

Neutral     93
Negative     4
Positive     3
Name: Sentiment, dtype: int64

In [38]:
tweets_df_nega = tweets_df.loc[tweets_df['Sentiment']=='Neutral']
tweets_df_nega.iloc[10].text

'godsunchained  looking for players join our growing card game community. our company is currently expanding and developing in thailand. there is no cost to sign up  website:  blockchain binance '

# Textblob

In [ ]:
%pip install -U textblob

In [ ]:
from textblob import TextBlob

def getSubjectivity(txt):
    return TextBlob(txt).sentiment.subjectivity
tweets_df['subjectivity'] = tweets_df.text.apply(getSubjectivity)
def getPolarity(txt):
    return TextBlob(txt).sentiment.polarity
tweets_df['polarity'] = tweets_df.text.apply(getPolarity)

tweets_df.head()

In [ ]:

tweets_df['sentiment_2'] = tweets_df.polarity.apply(lambda score:  "Negative" if score<0 else "Neutral" if score==0 else "Positive")
print(tweets_df['sentiment'].value_counts())
tweets_df.head(20)

In [ ]:
tweets_df['sentiment_2'].value_counts().to_dict()